# Custom Grid Topology

This notebook will showcase how to create UXarray data structures from custom grid topology files.

In [1]:
import uxarray as ux
import xarray as xr
import numpy as np

## Fixed Topology Example

In [2]:
node_lon = np.array([-20.0, 0.0, 20.0])
node_lat = np.array([-10.0, 10.0, -10.0])
face_node_connectivity = np.array(
    [
        [0, 1, 2],
    ]
)

In [3]:
uxgrid_tri = ux.Grid.from_topology(
    node_lon=node_lon, node_lat=node_lat, face_node_connectivity=face_node_connectivity
)

In [43]:
uxgrid_tri.plot(title="Triangle")

:Path   [Longitude,Latitude]

## Mixed Topology Example

In [50]:
node_lon = np.array([-20.0, 0.0, 20.0, -20, -40])
node_lat = np.array([-10.0, 10.0, -10.0, 10, -10])
face_node_connectivity = np.array([[0, 1, 2, -1], [0, 1, 3, 4]])

In [51]:
uxgrid_tri_quad = ux.Grid.from_topology(
    node_lon=node_lon,
    node_lat=node_lat,
    face_node_connectivity=face_node_connectivity,
    fill_value=-1,
)

In [52]:
uxgrid_tri_quad.plot(title="Triangle & Quad")

:Path   [Longitude,Latitude]

## Working with Existing Xarray Structures

### `xr.DataArray` to `ux.UxDataArray`

In [12]:
xrda_temp = xr.DataArray(
    name="temp",
    data=np.array(
        [
            [0, 1, 2, 3],
        ]
    ).T,
    dims=["time", "cell"],
)
xrda_temp

<xarray.DataArray 'temp' (time: 4, cell: 1)> Size: 32B
array([[0],
       [1],
       [2],
       [3]])
Dimensions without coordinates: time, cell

In [13]:
ugrid_dims = {"cell": "n_face"}

In [14]:
uxgrid = uxgrid_tri

In [16]:
uxda_temp = ux.UxDataArray.from_xarray(xrda_temp, uxgrid, ugrid_dims)
uxda_temp

<xarray.UxDataArray 'temp' (time: 4, n_face: 1)> Size: 32B
array([[0],
       [1],
       [2],
       [3]])
Dimensions without coordinates: time, n_face

### `xr.Dataset` to `ux.UxDataset`

In [17]:
# ds = xr.Dataset(data_vars={"temp": temp})
# ds

<xarray.Dataset> Size: 32B
Dimensions:  (time: 4, cell: 1)
Dimensions without coordinates: time, cell
Data variables:
    temp     (time, cell) int64 32B 0 1 2 3

In [18]:
ugrid_dims = {"cell": "n_face"}

In [19]:
uxgrid = uxgrid_tri

In [20]:
# uxds = ux.UxDataset.from_xarray(ds=ds, uxgrid=uxgrid_tri, ugrid_dims=ugrid_dims)
# uxds

<xarray.UxDataset> Size: 32B
Dimensions:  (time: 4, n_face: 1)
Dimensions without coordinates: time, n_face
Data variables:
    temp     (time, n_face) int64 32B 0 1 2 3